# **Importacion de paquetes**
---

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score,accuracy_score,confusion_matrix
from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings("ignore")

Entender que producto hay o no segun agrupaciones, por ejemplo grupos de cantidades de renta, id_segmento(universitario, no etx)

# **Preprocesamiento**
---

In [49]:
data_unida = pd.read_csv('../../Data/dataset_para_modelar.csv')

Sustituimos los valores anómalos

In [23]:
data_unida.replace(' NA', -1, inplace=True)
data_unida.replace('         NA', -1, inplace=True)
data_unida.fillna(-1, inplace=True)

Diccionarios para mapear:

In [24]:
emp_dict = {'N':0,-1:-1,'A':1,'B':2,'F':3,'S':4}
indata_unidaall_dict = {'N':0,-1:-1,'S':1}
sexo_dict = {'V':0,'H':1,-1:-1}
tiprel_dict = {'A':0,-1:-1,'I':1,'P':2,'N':3,'R':4}
indresi_dict = {'N':0,-1:-1,'S':1}
indext_dict = {'N':0,-1:-1,'S':1}
conyuemp_dict = {'N':0,-1:-1,'S':1}
segmento_dict = {-1:4,'01 - TOP':1,'02 - PARTICULARES':2,'03 - UNIVERSITARIO':3}

Mapeo de valores y conversion de los tipos

In [25]:
#Usamos int8 para datos numericos pequeños y 32 para mas elevados
data_unida['cod_persona'] = data_unida['cod_persona'].astype(np.int32)
data_unida['imp_renta'] = data_unida['imp_renta'].astype(np.int32)
#sustituimos el 99 por 2 para que sean valores mas cercanos
data_unida['xti_rel'] = data_unida['xti_rel'].map(lambda x: 2 if x == 99 else x).astype(np.int8)

data_unida['xti_empleado'] = data_unida['xti_empleado'].map(lambda x: emp_dict[x]).astype(np.int8)
data_unida['sexo'] = data_unida['sexo'].map(lambda x: sexo_dict[x]).astype(np.int8)
data_unida['edad'] = data_unida['edad'].astype(np.int16)

In [26]:
data_unida['xti_nuevo_cliente'] = data_unida['xti_nuevo_cliente'].astype(np.int8)
data_unida['num_antiguedad'] = data_unida['num_antiguedad'].map(lambda x: -1 if x == '     NA' else x).astype(int)
data_unida['num_antiguedad'] = data_unida['num_antiguedad'].astype(np.int16)
data_unida['xti_rel_1mes'] = data_unida['xti_rel_1mes'].map(lambda x: -2 if x == 'P' else x).astype(np.float16)
data_unida['xti_rel_1mes'] = data_unida['xti_rel_1mes'].astype(np.int8)

In [27]:
data_unida['tip_rel_1mes'] = data_unida['tip_rel_1mes'].map(lambda x: tiprel_dict[x]).astype(np.int8)
data_unida['indresi'] = data_unida['indresi'].map(lambda x: indresi_dict[x]).astype(np.int8)
data_unida['indext'] = data_unida['indext'].map(lambda x: indext_dict[x]).astype(np.int8)

Convertimos des canal a numerico

In [28]:
canal_dict={value:idx+1  for idx, value in enumerate(data_unida['des_canal'].unique())}
data_unida['des_canal'] = data_unida['des_canal'].map(lambda x: canal_dict[x]).astype(np.int16)

In [29]:
pais_dict={key:id+1  for id,key in enumerate(data_unida['pais'].unique())}

In [30]:
data_unida['xti_extra'] = data_unida['xti_extra'].map(lambda x: indata_unidaall_dict[x]).astype(np.int8)
data_unida['pais'] = data_unida['pais'].map(lambda x: pais_dict[x]).astype(np.int8)
data_unida['tip_dom'] = data_unida['tip_dom'].astype(np.int8)
data_unida['cod_provincia'] = data_unida['cod_provincia'].astype(np.int8)

In [31]:
data_unida['xti_actividad_cliente'] = data_unida['xti_actividad_cliente'].astype(np.int8)
data_unida['fecha_dato_month'] = data_unida['mes'].map(lambda x: int(x[5:7])).astype(np.int8)
data_unida['fecha_dato_year'] = data_unida['mes'].map(lambda x: int(x[0:4]) - 2015).astype(np.int8)
data_unida['month_int'] = (data_unida['fecha_dato_month'] + 12 * data_unida['fecha_dato_year']).astype(np.int8)
data_unida.drop('mes',axis=1,inplace=True)

In [32]:
data_unida['fecha1'] = data_unida['fecha1'].map(lambda x: '2020-01-01' if x == -1 else x)
data_unida['fecha_alta_month'] = data_unida['fecha1'].map(lambda x: int(x[5:7])).astype(np.int16)
data_unida['fecha_alta_year'] = data_unida['fecha1'].map(lambda x: int(x[0:4]) - 1995).astype(np.int16)

In [33]:
data_unida['fecha_alta_day'] = data_unida['fecha1'].map(lambda x: int(x[8:10])).astype(np.int16)
data_unida['fecha_alta_month_int'] = (data_unida['fecha_alta_month'] + 12 * data_unida['fecha_alta_year']).astype(np.int16)
data_unida['fecha_alta_day_int'] = (data_unida['fecha_alta_day'] + 30 * data_unida['fecha_alta_month'] + 365 * data_unida['fecha_alta_year']).astype(np.int32)
data_unida.drop('fecha1',axis=1,inplace=True)

In [34]:
data_unida['fec_ult_cli_1t'] = data_unida['fec_ult_cli_1t'].map(lambda x: '2020-01-01' if x == -1 else x)
data_unida['ult_fec_cli_1t_month'] = data_unida['fec_ult_cli_1t'].map(lambda x: int(x[5:7])).astype(np.int16)
data_unida['ult_fec_cli_1t_year'] = data_unida['fec_ult_cli_1t'].map(lambda x: int(x[0:4]) - 2015).astype(np.int16)
data_unida['ult_fec_cli_1t_day'] = data_unida['fec_ult_cli_1t'].map(lambda x: int(x[8:10])).astype(np.int16)
data_unida['ult_fec_cli_1t_month_int'] = (data_unida['ult_fec_cli_1t_month'] + 12 * data_unida['ult_fec_cli_1t_year']).astype(np.int8)
data_unida.drop('fec_ult_cli_1t',axis=1,inplace=True)

In [35]:
data_unida['id_segmento'].unique()

array(['02 - PARTICULARES', '03 - UNIVERSITARIO', '01 - TOP', -1],
      dtype=object)

In [36]:
data_unida['id_segmento'] = data_unida['id_segmento'].map(lambda x: segmento_dict[x]).astype(np.int8)
target_cols=[f'ind_prod{i}' for i in range(1,26)]
for col in target_cols:
    data_unida[col] = data_unida[col].astype(np.int8) #Reducimos el tamaño porque son binarios y asi va mas rapido

Obtenemos un diccionario con la cantidad de veces que se han comprado cada producto en funcion de cada categoria de id_segmento, para posteriormente aquellos productos que para un id_segmento especifico haya habido 0 compras no recomendarselo a nuevos usuarios que pertenezcan a dicho segmento y crear modelos unicamente para los productos donde si ha habido compras previas.

In [37]:
#Obtenemos los productos que si se han comprado por cada segmento
diccionario_segmentos=data_unida.groupby(by=['id_segmento'])[target_cols].sum().T.to_dict()
diccionario_seg_proc={}
for key,value in diccionario_segmentos.items():
    sol=[]
    for prod,res in value.items():
        if res>0:
            sol.append(prod)
    diccionario_seg_proc[key]=sol
diccionario_seg_proc

{1: ['ind_prod3',
  'ind_prod4',
  'ind_prod5',
  'ind_prod7',
  'ind_prod8',
  'ind_prod9',
  'ind_prod10',
  'ind_prod11',
  'ind_prod12',
  'ind_prod13',
  'ind_prod14',
  'ind_prod15',
  'ind_prod16',
  'ind_prod17',
  'ind_prod18',
  'ind_prod19',
  'ind_prod20',
  'ind_prod21',
  'ind_prod22',
  'ind_prod23',
  'ind_prod24',
  'ind_prod25'],
 2: ['ind_prod1',
  'ind_prod3',
  'ind_prod4',
  'ind_prod5',
  'ind_prod6',
  'ind_prod7',
  'ind_prod8',
  'ind_prod9',
  'ind_prod10',
  'ind_prod11',
  'ind_prod12',
  'ind_prod13',
  'ind_prod14',
  'ind_prod15',
  'ind_prod16',
  'ind_prod17',
  'ind_prod18',
  'ind_prod19',
  'ind_prod20',
  'ind_prod21',
  'ind_prod22',
  'ind_prod23',
  'ind_prod24',
  'ind_prod25'],
 3: ['ind_prod3',
  'ind_prod4',
  'ind_prod5',
  'ind_prod7',
  'ind_prod8',
  'ind_prod9',
  'ind_prod10',
  'ind_prod11',
  'ind_prod12',
  'ind_prod13',
  'ind_prod14',
  'ind_prod16',
  'ind_prod17',
  'ind_prod18',
  'ind_prod19',
  'ind_prod20',
  'ind_prod22',
 

In [38]:
data_unida.head()

,Unnamed: 0,cod_persona,pais,sexo,edad,xti_empleado,xti_nuevo_cliente,num_antiguedad,xti_rel,xti_rel_1mes,...,month_int,fecha_alta_month,fecha_alta_year,fecha_alta_day,fecha_alta_month_int,fecha_alta_day_int,ult_fec_cli_1t_month,ult_fec_cli_1t_year,ult_fec_cli_1t_day,ult_fec_cli_1t_month_int
0,0,178103,1,1,35,0,0,6,1,1,...,1,1,20,12,241,7342,1,5,1,61
1,1,503082,1,0,27,0,0,35,1,1,...,1,8,17,10,212,6455,1,5,1,61
2,2,502996,1,0,37,0,0,35,1,1,...,1,8,17,10,212,6455,1,5,1,61
3,3,503053,1,1,23,0,0,35,1,1,...,1,8,17,10,212,6455,1,5,1,61
4,4,503031,1,1,44,0,0,35,1,1,...,1,8,17,10,212,6455,1,5,1,61


Guardamos el dataset con el progreso de limpieza inicial

In [39]:
data_unida.to_csv('sant_limpio.csv',index=False)

In [40]:
data_unida.rename(columns={'cod_persona':'id'},inplace=True)

Analizamos los productos con menor porcentage de ventas (con menos 1).

In [41]:
cols_valor_unico=[]
muy_bajos_unos=[]
bajos_unos=[]
medios_unos=[]
for col in target_cols:
    if len(data_unida[col].value_counts().values)<2:
        cols_valor_unico.append(col)
    elif data_unida[col].value_counts().values[1]/len(data_unida) <0.001:
        muy_bajos_unos.append(col)
    elif data_unida[col].value_counts().values[1]/len(data_unida) <0.005:
        bajos_unos.append(col)
    else:
        medios_unos.append(col)
print('Columnas de valor unico: ',cols_valor_unico)
print('Columnas con muy poco : ',muy_bajos_unos)
print('Columnas con pocos : ',bajos_unos)
print('Columnas con intermedio : ',medios_unos)

Columnas de valor unico:  ['ind_prod2']
Columnas con muy poco :  ['ind_prod1', 'ind_prod4']
Columnas con pocos :  ['ind_prod10', 'ind_prod11', 'ind_prod17', 'ind_prod21']
Columnas con intermedio :  ['ind_prod3', 'ind_prod5', 'ind_prod6', 'ind_prod7', 'ind_prod8', 'ind_prod9', 'ind_prod12', 'ind_prod13', 'ind_prod14', 'ind_prod15', 'ind_prod16', 'ind_prod18', 'ind_prod19', 'ind_prod20', 'ind_prod22', 'ind_prod23', 'ind_prod24', 'ind_prod25']


Al principio del trabajo pensamos en que al haber varias observciones del mismo indiviudo en distintas fechas era buena idea hacer series temporales. Tras al hacer una prueba con redes neuronales (tratandolo como series temporales) y otra con una clasificación (usando sólamente las últimas fechas) vimos que la diferencia era insignificante, por lo que decidimos quedarnos con la idea de clasificación al ser más sencilla.

Decidimos entonces añadir columnas con la informacion de las compras del N periodo pasado, en este caso 1, de esta forma conservamos algo de informacion historica sin aumentar considerablemente el tamaño del dataset con tecnicas como por ejemplo la generacion de ventana.

La idea principal era realizar el proceso completo para diferentes N periodos por ejemplo de 1 a 5 y hacer una media de las predicciones de los diferentes modelos, sin embargo, supone un coste computacional que no podemos afrontar. Consideramos igualmente que de esta forma mejoraria la robustez de la solucion.

Hacemos el proceso de aplanamiento del dataset sobre el dataset completo. Este paso consiste en: de las 16 observaciones que puede tener un individuo como máximo, las dejamos en una única fila, escogiendo la última y penúltima fecha (mes16 y mes15 respectivamnete)

In [42]:

cols_to_combine = ['edad', 'num_antiguedad', 'des_canal', 'cod_provincia',
       'fecha_alta_day', 'fecha_alta_month', 'fecha_alta_month_int','fecha_alta_day_int',
       'fecha_alta_year', 'fecha_dato_month', 'fecha_dato_year',
       'xti_actividad_cliente',
       'xti_empleado',
       'xti_nuevo_cliente',
       'indext',
       'xti_extra', 'xti_rel', 'xti_rel_1mes', 'indresi',
       'pais', 'imp_renta', 'id_segmento', 'sexo',
       'tip_rel_1mes', 'ult_fec_cli_1t_day', 'ult_fec_cli_1t_month',
       'ult_fec_cli_1t_month_int', 'ult_fec_cli_1t_year']

#Creamos columnas con un shift sobre el dataset inicial (Idea de competicion Kaggle)
DIFF_CONDS = {}
for shift_val in [1]:
    name = 'id_shift_' + str(shift_val)
    data_unida[name] = data_unida['id'].shift(shift_val).fillna(0).astype(np.int32)
    DIFF_CONDS[shift_val] = ((data_unida['id'] - data_unida[name]) != 0)
    data_unida.drop(name,axis = 1,inplace=True)
shifted_feature_names = []
for col in cols_to_combine + target_cols:
    for shift_val in [1]:
        name = col + '_s_' + str(shift_val)
        data_unida[name] = data_unida[col].shift(shift_val).fillna(0).astype(np.int32)
        data_unida[name][DIFF_CONDS[shift_val]] = 0
        if col in cols_to_combine:
            shifted_feature_names.append(name)

In [43]:
data_unida=data_unida.iloc[:,1:]

Creamos columnas con las variacion de las X variable para predecir respecto al periodo previo.

In [44]:
#Idea sacada de kaggle
diff_feautres_s1 = []
for col in cols_to_combine:
    name = col + '_s1_diff'
    diff_feautres_s1.append(name)
    data_unida[name] = (data_unida[col] - data_unida[col + '_s_1']).astype(np.int32)

Hacemos feature engineering para obtener más estadísticos de nuestros datos.

In [45]:
#Feature engineering de mes,antiguedad, edad y renta
MIN_MONTH_DICT = data_unida.groupby('id')['month_int'].min().to_dict()
data_unida['min_month_int'] = data_unida['id'].map(lambda x: MIN_MONTH_DICT[x]).astype(np.int8)

#Antiguedad minima del usuario
MIN_ANTIGUEDAD_DICT = data_unida.groupby('id')['num_antiguedad'].min().to_dict()
data_unida['min_antiguedad'] = data_unida['id'].map(lambda x: MIN_ANTIGUEDAD_DICT[x]).astype(np.int16)

#Antiguedad maxima del usuario
MAX_ANTIGUEDAD_DICT = data_unida.groupby('id')['num_antiguedad'].max().to_dict()
data_unida['max_antiguedad'] = data_unida['id'].map(lambda x: MAX_ANTIGUEDAD_DICT[x]).astype(np.int16)

#Edad minima del usuario
MIN_AGE_DICT = data_unida.groupby('id')['edad'].min().to_dict()
data_unida['min_edad'] = data_unida['id'].map(lambda x: MIN_AGE_DICT[x]).astype(np.int16)

#Edad maxima del usuario
MAX_AGE_DICT = data_unida.groupby('id')['edad'].max().to_dict()
data_unida['max_edad'] = data_unida['id'].map(lambda x: MAX_AGE_DICT[x]).astype(np.int16)

#Minimo maximo y desviancion estandar de la renta por cada usuario
MIN_RENTA_DICT = data_unida.groupby('id')['imp_renta'].min().to_dict()
data_unida['min_renta'] = data_unida['id'].map(lambda x: MIN_RENTA_DICT[x])
MAX_RENTA_DICT = data_unida.groupby('id')['imp_renta'].max().to_dict()
data_unida['max_renta'] = data_unida['id'].map(lambda x: MAX_RENTA_DICT[x])

Ordenamos el dataset por persona y periodo para despues eliminar los duplicados y quedarnos asi con el ultimo periodo de cada usuario.

In [46]:
RENTA_VAL_COUNTS = data_unida.groupby('imp_renta')['id'].nunique().to_dict()
data_unida['renta_freq'] = data_unida['imp_renta'].map(lambda x: RENTA_VAL_COUNTS[x])
data_unida.sort_values(by = ['id','month_int'],inplace=True)
#Eliminamos los id duplicados porque hemos reducido la informacion a una sola fila por usuario
combined_nd = data_unida.drop_duplicates('id')

Bucle para en funcion del segmento recuperar los productos que aplican y sobre estos crear un modelo Catboost para predecir dicho producto y en caso de los productos que no aplican retornamos una lista de 0.

In [47]:
otros=True
columnas_sin_target=combined_nd.columns
columnas_sin_target=[col for col in columnas_sin_target if col not in target_cols]
resultados={}
for segment in combined_nd['id_segmento'].unique():
    res_seg={}
    acc=[]
    rec=[]
    models=[]
    for prod in diccionario_seg_proc[segment]:
        if prod!='ind_prod2':
            X=combined_nd[columnas_sin_target].copy()
            y=combined_nd[prod].copy()
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

            # Identificar características categóricas
            categorical_features_indices = np.where(X.dtypes != np.float64)[0]

            # Calcular los pesos de las clases para tratar de corregir el desbalanceo
            class_weights = [len(y) / (2 * sum(y == c)) for c in np.unique(y)]
            # Crear el modelo CatBostClassifier
            modelo = CatBoostClassifier(
                depth=6,
                loss_function='Logloss',
                class_weights=class_weights,
                verbose=1000)

            # Entrenar el modelo
            modelo.fit(X_train, y_train)

            y_pred = modelo.predict_proba(X_test)
            threshold=.5
            y_pred=[1 if p[1]>threshold else 0 for p in y_pred ]
            models.append(modelo)
            # Evaluar la precisión del modelo
            accuracy = accuracy_score(y_test, y_pred)
            acc.append(accuracy)
            print('\n','*'*50)
            print('\n PARA EL SEGMENTO: ',segment)
            print('\n','*'*50)
            print('-'*50,'\nProducto: ',prod)
            print(f'Accuracy: {accuracy}')
            recall = recall_score(y_test, y_pred)
            rec.append(recall)
            print(f'Recall: {recall}')
            conf = confusion_matrix(y_test, y_pred)
            print(f'Matriz de confusion: {conf} ','\n')
        if otros:
            faltan=25-len(acc)
            if faltan>0:
                acc+=[1]*faltan
                rec+=[1]*faltan
    resultados[segment]={'accuracy':acc,'recall':rec,'models':models}


Learning rate set to 0.04838
0:	learn: 0.6354288	total: 90.2ms	remaining: 1m 30s


KeyboardInterrupt: 

Mostramos los valore medios de Accuracy y Recall por cada segmento.

In [ ]:
#Media de los resultados
for segment in resultados.keys():
    print('Valores medios por segmento: ',segment)
    print('Accuracy medio: ', np.mean(resultados[segment]['accuracy']))
    print('Recall medio: ',np.mean(resultados[segment]['recall']))

**Resultados sin filtrado por segmento:**
- Accuracy medio:  0.9219796565483825
- Recall medio:  0.564505857250443
---

**Valores medios por segmento:  4**

- Accuracy medio:  0.8971207287795573
- Recall medio:  0.7402386002440919

**Valores medios por segmento:  2**

- Accuracy medio:  0.9217970642724812
- Recall medio:  0.5633717445733596

**Valores medios por segmento:  3**

- Accuracy medio:  0.9070382642154767
- Recall medio:  0.6177127601546981

**Valores medios por segmento:  1**

- Accuracy medio:  0.9147022814722686
- Recall medio:  0.5691328122618469

In [ ]:
import os

Guardamos los modelos creados para evitar tener que regenerarlos cada vez.

In [ ]:
for segment in combined_nd['id_segmento'].unique():
    os.mkdir(f'Models/{segment}')
    for prods,models  in zip(diccionario_seg_proc[segment],resultados[segment]['models']):
        raiz=f'Models/{segment}/{prods}.cb'
        models.save_model(raiz)

En la siguiente celda esta el código para recuperar todos los modelos creados.

In [ ]:
import os
modelos={}
for segment in combined_nd['id_segmento'].unique():
    lista_mods=[]
    for model in os.listdir(f'Models/{segment}'):
        raiz=f'Models/{segment}/{model}'
        cat = CatBoostClassifier()
        cat.load_model(raiz)
        lista_mods.append(cat)
    modelos[segment]=lista_mods

Adaptamos la última predicción para sustituir el histórico por la última observación ya que queremos recomendar para la siguiente.

In [ ]:
prods_anterior=[f'ind_prod{i}_s_1' for i in range(1,26)]
data_para_predecir=combined_nd.copy()
data_para_predecir.reset_index(inplace=True)
for i in range(1,26):
    data_para_predecir[f'ind_prod{i}_s_1']=data_para_predecir[f'ind_prod{i}']


Calculamos la predicción para cada persona en funciòn de su segmento y de si el producto ha sido descartado para dicho segmento.

In [ ]:
#Calculamos la prediccion de cada persona en funcion de su segmento, si en el segmento se ha considerado que un producto no se predice ponemos un 0
predicciones_sin_procesar={}
for segment in data_para_predecir['id_segmento'].unique():
    predicciones={}
    X=data_para_predecir[columnas_sin_target].copy()
    for idx,prod in enumerate(diccionario_seg_proc[segment]):
        model=modelos[segment][idx]
        res=model.predict_proba(X)
        predicciones[prod]=res

    for prod in set(set(target_cols)).difference(diccionario_seg_proc[segment]):
        predicciones[prod]=np.zeros(X.shape[0])
    predicciones_sin_procesar[segment]=predicciones

Recuperamos el valor de cada predicción correspondiente a cada persona para después poder procesarlo mas facilmente.

In [ ]:
p={}
for persona in data_para_predecir['id'].unique():
    data=data_para_predecir[data_para_predecir['id']==persona]
    indice_persona=data.index[0]
    for key, value in predicciones_sin_procesar.items():
        if data['id_segmento'].values[0]==key:
            seg_pred=predicciones_sin_procesar[key]
            soluciones={}
            for k,v in seg_pred.items():
                if isinstance(v[indice_persona],np.ndarray): #Si es un array devolvemos el segundo elemento que es la probabilidad de la clase 1
                    soluciones[k]=v[indice_persona][1]
                else:
                    soluciones[k]=v[indice_persona]
    p[persona]=soluciones


Convertimos los resultados de las predicciones a un Dataframe para facilitar el procesamiento posterior de los resultados.

In [ ]:
#Convertimos las predicciones a un df
columnas=target_cols + ['id']
df_final = pd.DataFrame(columns=columnas)
for id,k in enumerate(p.keys()):
    dt={j:[v] for j,v in p[k].items()}
    dt['id']=k
    predicciones_df=pd.DataFrame(dt,columns=columnas)
    df_final=pd.concat([df_final,predicciones_df],axis=0)
df_final.head()

Guardamos el dataframe con las predicciones del modelo.

In [ ]:
df_final.to_csv('pruebita.csv',index=False)

In [ ]:
data=pd.read_csv('pruebita.csv')

Recuperamos la informacion del periodo previo al predicho para tal y como se nos ha indicado, no recomendar productos que el usuario ya tenia en el periodo anterior.

In [ ]:
targets_prev=[f'ind_prod{i}_s_1' for i in range(1,26)]
data[targets_prev]=data_para_predecir[target_cols]

In [ ]:
for i in range(25):
    # Para verificar si el valor es 0 y asignar 0 en la columna correspondiente
    data.loc[data[targets_prev[i]] == 1, target_cols[i]] = 0

Eliminammos los valores historicos ya que no los tenemos que enviar en la prediccion.

In [ ]:
data.drop(targets_prev,axis=1,inplace=True)

In [ ]:
data.to_csv('Predicciones_correctas_sin_procesar.csv',index=False)

In [ ]:
data=pd.read_csv('Predicciones_correctas_sin_procesar.csv')

Creamos una funcion para extraer los productos a recomendar en formato de lista siguiendo el orden obtenido a partir de las probabilidades del modelo.

In [ ]:
def row_to_ordered_dict(row):
    row_dict = row.to_dict()
    sorted_dict = dict(sorted(row_dict.items(), key=lambda item: item[1], reverse=True))
    return sorted_dict

# Crear una nueva columna con los diccionarios ordenados
data['predicted'] = data.apply(row_to_ordered_dict, axis=1)

In [ ]:
data['predicted']= data['predicted'].apply(lambda x: [k for k,val in x.items() if (val>.5) & (k!='id')] )

In [ ]:
data.rename(columns={'id':'cod_persona'},inplace=True)

Guardamos nuestras predicciones con las probabilidades.

In [ ]:
data.to_csv('predicciones.csv',index=False)

In [ ]:
for i in range(25):
    data.loc[data[target_cols[i]] >= .5,target_cols[i]]=1
    data.loc[data[target_cols[i]] < .5,target_cols[i]]=0

Guardamos nuestras recomendaciones en formato 0 si no recomendamos y 1 si si recomendamos.

In [ ]:
data.to_csv('soluciones.csv',index=False)